# Analysis notebook for CAB

In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
import pymongo as pm
import os, sys

In [2]:
# display all columns
pd.set_option('display.max_columns', None)

## Create folders to save out results

In [3]:
## create relevant project subdirs
proj_dir = os.path.abspath('..')
analysis_dir =  os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
csv_dir = os.path.join(results_dir,'csv')
plots_dir = os.path.join(results_dir,'plots')

def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return

In [4]:
print("The project directory is: {}".format(proj_dir))

The project directory is: /Users/felixbinder/Cloud/Grad School/Fan Lab/BACH/cognitive-ai-benchmarking


In [5]:
makedir(results_dir)
makedir(csv_dir)
makedir(plots_dir)

In [6]:
# we need to get to the repo level to import cabutils
sys.path.append(proj_dir)
sys.path.append(os.path.join(proj_dir,'stimuli'))
import cabutils
from stimuli.experiment_config import *

## Experiment parameters

In [7]:
PROJECT = "Physion_V1_5" 
DATASET = "Dominoes"
TASK = "OCP"
ITERATION = "pilot_1"
EXPERIMENT = DATASET + "_" + TASK

## Load data from database

In [8]:
!ssh -fNL 27017:127.0.0.1:27017 fbinder@cogtoolslab.org

bind [127.0.0.1]:27017: Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 27017
Could not request local forwarding.


In [9]:
## set the relevant variables
dbname = PROJECT+'_output' ## which database are we using, e.g. BACH?
colname = EXPERIMENT ## which collection inside this database, e.g. dominoes?
iterationName = ITERATION ## which iterations do we want to analyze, e.g. iter1?

# establish connection to mongo
conn = get_db_connection()
db = conn[dbname] 
coll = db[colname]
## fetch all records from this iterationName
print("Reading results. This might take a while...")
K = coll.find({'iterationName':iterationName})
li = list(K)
_M = pd.DataFrame(li)
print("Read {} records from the database. Done.".format(len(_M)))

Checking database connection...
Reading results. This might take a while...
Read 3479 records from the database. Done.


## Prolific fixes
A bug in prolific means that we have read the Prolific from a manual field rather than the URL field

In [10]:
def fill_ProlificIDs(df):
    for game_id in tqdm(df['gameID'].unique()):
        m = df['gameID'] == game_id
        if set(df[m]['prolificID'].unique()) == set([None]):
            # no prolific ID recorded. Can we recover it?
            responses = df[m]['responses'].dropna()
            pID = None # default value is None
            for r in responses:
                if "manual_prolificID" in r:
                    try:
                        r = eval(r) # really bad idea to run eval on data from participants TODO make safe
                    except:
                        continue
                    pID = r['manual_prolificID']
            df.loc[m,'prolificID'] = pID
    return df


In [11]:
_M = fill_ProlificIDs(_M)

100%|██████████| 38/38 [00:00<00:00, 527.90it/s]


In [12]:
_M = _M[_M['prolificID'] != "TEST"] #exclude manual tests

## Apply exclusion criteria

In [13]:
_M['prolificID'].value_counts()

62cee0344bbf495f81b5643a    155
5e6473fbc15dbe1f71eea95b    155
5a9a6f5f35237b000112920f    155
55fb068b77f33b0005cb1efc    155
5e13859753e8469d270982c0    155
sf639c21192b540a86e6e559    155
5460ef52fdf99b49ea4b5906    155
5ad1f9914d061300011b1173    155
5c144c641f6f1500014950e5    155
6347200d8ded5172ec43e0bb    155
Name: prolificID, dtype: int64

In [14]:
## get unique gameIDs for completed games only
completed_games = []
for gameID in _M['gameID'].unique():
    ## check if we have a survey event for the gameID
    events = list(_M[_M['gameID'] == gameID]['eventType'])
    prolificIDs = list(_M[_M['gameID'] == gameID]['prolificID'])
    if ("survey" in events) & (set(prolificIDs) != set([None])): completed_games.append(gameID)

print("We have",len(completed_games),"completed unique games.")

We have 10 completed unique games.


In [15]:
## filter on complete games 
M = _M[_M['gameID'].isin(completed_games)]
print("We're left with {} games".format(M['gameID'].nunique()))

We're left with 10 games


In [19]:
M

,_id,rt,stimulus,response,trial_type,trial_index,time_elapsed,internal_node_id,gameID,stims_not_preloaded,type,iterationName,inputid,response_allowed_while_playing,condition,prompt,choices,bounciness,color,distractors,dynamic_friction,git_commit,initial_position,initial_rotation,mass,model_names,object_ids,object_segmentation_colors,occluders,probe_id,probe_mass,probe_type,push_time,randomize,remove_middle,room,scale,scale_x,scale_y,scale_z,seed,static_friction,stimulus_name,target_id,target_rotation,target_type,trial_num,trial_seed,zone_id,middle_objects,middle_type,target_contacting_zone,target_hit_zone_label,does_target_contact_zone,maps_url,mp4s_url,hdf5_url,trialNum,overlay,overlay_time,blink_time,stop,width,height,post_trial_gap,prolificID,studyID,sessionID,stim_ID,button_html,autoplay,controls,start,rate,trial_ends_after_video,trial_duration,margin_vertical,margin_horizontal,response_ends_trial,correct,projName,expName,stimulus_metadata,workerId,hitID,aID,eventType,responses,inputID,question_order
1853,63c9fd41dacc9212d946d7f3,6046.9,[https://physion-v1-5-dominoes-pilot-1.s3.amaz...,YES,video-overlay-button-response,1,24748,0.0-1.0,7400-2698c32b-f73b-417a-b72c-2b5d565c9cf9,False,video-overlay-button-response,pilot_1,63c18b25082622779f6e2743,False,familiarization_prediction,Is the red object going to hit the yellow area?,"[YES, NO]","[0.4, 0.4, 0.4, 0.4]","[[0.8362804732356993, 0.05309902108038478, 0.3...",[atmosphere_nodo],"[0.25, 0.25, 0.25, 0.25]",,"[[1.100000023841858, 0, 0], [0.5, 0, 0], [-0.5...","[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 18.40457...","[0.04999999999999999, 0.06249999999999999, 0.0...","[cube, cube, cube, cube, atmosphere_nodo, coff...","[1, 2, 3, 4, 5, 6]","[[131, 70, 184], [76, 3, 38], [122, 205, 218],...",[coffee_0023],3,2,cube,15,0,False,mm_kitchen_1b,"[[0.5, 0.009999999776482582, 2], [0.1000000014...","[0.5, 0.1, 0.1, 0.1, 2.5543558939089883, 1.339...","[0.01, 0.5, 0.5, 0.5, 2.5543558939089883, 1.33...","[2, 0.25, 0.25, 0.25, 2.5543558939089883, 1.33...",178,"[0.4, 0.4, 0.4, 0.4]",pilot_dominoes_1mid_J025R45_boxroom_0000,2,"[0, 0, 0]",cube,0,178000,1,[cube],cube,False,False,False,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,17,[https://physion-v1-5-dominoes-pilot-1.s3.amaz...,3,500,0.45,500,500,0,62cee0344bbf495f81b5643a,None,None,pilot_dominoes_1mid_J025R45_boxroom_0000,"<button class=""jspsych-btn"">%choice%</button>",False,False,0,1,False,None,0px,8px,True,False,Physion_V1_5,Dominoes_OCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1854,63c9fd49dacc9212d946d7f4,5129.9,[https://physion-v1-5-dominoes-pilot-1.s3.amaz...,NO,video-overlay-button-response,3,32476,0.0-3.0,7400-2698c32b-f73b-417a-b72c-2b5d565c9cf9,False,video-overlay-button-response,pilot_1,63c18b25082622779f6e2743,False,familiarization_prediction,Is the red object going to hit the yellow area?,"[YES, NO]","[0.4, 0.4, 0.4, 0.4]","[[0.02785074736982751, 0.24662297125495825, 0....",[vase_laura_deko_vase_set],"[0.25, 0.25, 0.25, 0.25]",,"[[1.100000023841858, 0, 0], [0.5, 0, 0], [-0.5...","[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 7.978260...","[0.04999999999999999, 0.06249999999999999, 0.0...","[cube, cube, cube, cube, vase_laura_deko_vase_...","[1, 2, 3, 4, 5, 6]","[[127, 71, 34], [10, 28, 73], [125, 238, 106],...",[linbrazil_diz_armchair],3,2,cube,15,0,False,mm_kitchen_1b,"[[0.5, 0.009999999776482582, 2], [0.1000000014...","[0.5, 0.1, 0.1, 0.1, 2.015186445049896, 0.5328...","[0.01, 0.5, 0.5, 0.5, 2.015186445049896, 0.532...","[2, 0.25, 0.25, 0.25, 2.015186445049896, 0.532...",178,"[0.4, 0.4, 0.4, 0.4]",pilot_dominoes_1mid_J025R45_boxroom_0006,2,"[0, 0, 0]",cube,6,178006,1,[cube],cube,False,False,False,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,https://physion-v1-5-dominoes-pilot-1.s3.amazo...,20,[https://physion-v1-5-dominoes-pilot-1.s3.amaz...,3,500,0.45,500,500,0,62cee0344bbf495f81b5643a,None

In [20]:

## separate into T (trials) and S (survey) dataframes
T_train = M[M['eventType']=='training_trials']
T_test = M[M['eventType']=='test_trials']
T = pd.concat([T_train, T_test], ignore_index=True, sort=False)
S = M[M['eventType']=='survey_data']

## make sure that all the games in T are also in S (sanity check)
Tgames = list(np.unique(T['gameID'].values))
Sgames = list(np.unique(S['gameID'].values))
assert len(np.intersect1d(Tgames,Sgames))==len(Tgames)

## Drop the sensitive information and save to csv locally

In [21]:
## ANONYMIZE DATAFRAMES: drop ProlificID & any other potentially identifying info from T & S
T = T.drop(columns=['prolificID'])
S = S.drop(columns=['prolificID'])

In [81]:
## save out to file 
S.to_csv(os.path.join(csv_dir, '{}_{}_survey.csv'.format(colname, iterationName)),index=False)
T.to_csv(os.path.join(csv_dir, '{}_{}_trials.csv'.format(colname, iterationName)),index=False)
print('Saved successfully to {}!'.format(os.path.join(csv_dir,'{}_{}_survey.csv'.format(colname, iterationName))))
print('Saved successfully to {}!'.format(os.path.join(csv_dir,'{}_{}_trials.csv'.format(colname, iterationName))))

Saved successfully to /Users/felixbinder/Cloud/Grad School/Fan Lab/BACH/cognitive-ai-benchmarking/results/csv/Dominoes_OCP_pilot_1_survey.csv!
Saved successfully to /Users/felixbinder/Cloud/Grad School/Fan Lab/BACH/cognitive-ai-benchmarking/results/csv/Dominoes_OCP_pilot_1_trials.csv!


# visualize the data and do the analysis you want

In [35]:
df = M[M['trial_type'] == 'video-overlay-button-response']
df = df[df['condition'] != "familiarization_prediction"]

In [37]:
df['correct'].value_counts()


True     1247
False     253
Name: correct, dtype: int64

In [38]:
# how many did we get correct?
df['correct'].value_counts()[True]/len(df)

0.8313333333333334